In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import WeightedRandomSampler
import torch.nn.functional as F
from torchvision import datasets, models, transforms

### Finding numbers of files in the training set with their labels

In [2]:
import os

APP_FOLDER = r"newimages"

totalFiles = 0
totalDir = 0
x=0
y=[]
for base, dirs, files in os.walk(APP_FOLDER):
    if (x==0):
        z=dirs
    totalFiles = 0
    for Files in files:
        totalFiles += 1
    Files=files
    print('number of files',totalFiles)
    if (x>0):
        y.append(totalFiles)
    x=x+1


number of files 0
number of files 72
number of files 185
number of files 252
number of files 16
number of files 214
number of files 188
number of files 196
number of files 166
number of files 164
number of files 239
number of files 168
number of files 195
number of files 187
number of files 158
number of files 132
number of files 82
number of files 118
number of files 14
number of files 56
number of files 218
number of files 182
number of files 187
number of files 188
number of files 196
number of files 54
number of files 200
number of files 232
number of files 134
number of files 102
number of files 140
number of files 180
number of files 162
number of files 172
number of files 152
number of files 160
number of files 185
number of files 138
number of files 116
number of files 197
number of files 202
number of files 202
number of files 197
number of files 196
number of files 174
number of files 178
number of files 90
number of files 114
number of files 100
number of files 122
number of

### making a dictionary of training files

In [3]:
m=0
cls_dic={}
for i in z:
    cls_dic[i]=y[m]
    m+=1
cls_dic

{'n02085620-Chihuahua': 72,
 'n02085782-Japanese_spaniel': 185,
 'n02085936-Maltese_dog': 252,
 'n02086079-Pekinese': 16,
 'n02086240-Shih-Tzu': 214,
 'n02086646-Blenheim_spaniel': 188,
 'n02086910-papillon': 196,
 'n02087046-toy_terrier': 166,
 'n02087394-Rhodesian_ridgeback': 164,
 'n02088094-Afghan_hound': 239,
 'n02088238-basset': 168,
 'n02088364-beagle': 195,
 'n02088466-bloodhound': 187,
 'n02088632-bluetick': 158,
 'n02089078-black-and-tan_coonhound': 132,
 'n02089867-Walker_hound': 82,
 'n02089973-English_foxhound': 118,
 'n02090379-redbone': 14,
 'n02090622-borzoi': 56,
 'n02090721-Irish_wolfhound': 218,
 'n02091032-Italian_greyhound': 182,
 'n02091134-whippet': 187,
 'n02091244-Ibizan_hound': 188,
 'n02091467-Norwegian_elkhound': 196,
 'n02091635-otterhound': 54,
 'n02091831-Saluki': 200,
 'n02092002-Scottish_deerhound': 232,
 'n02092339-Weimaraner': 134,
 'n02093256-Staffordshire_bullterrier': 102,
 'n02093428-American_Staffordshire_terrier': 140,
 'n02093647-Bedlington_ter

### sorting the files in the descending order of the number of images

In [4]:
sort={k: v for k, v in sorted(cls_dic.items(), key=lambda item: item[1],reverse=True)}

In [5]:
sort_key=list(sort.keys())
sort_key

['n02085936-Maltese_dog',
 'n02088094-Afghan_hound',
 'n02092002-Scottish_deerhound',
 'n02112018-Pomeranian',
 'n02090721-Irish_wolfhound',
 'n02107683-Bernese_mountain_dog',
 'n02111889-Samoyed',
 'n02086240-Shih-Tzu',
 'n02111500-Great_Pyrenees',
 'n02111129-Leonberg',
 'n02110806-basenji',
 'n02097474-Tibetan_terrier',
 'n02095889-Sealyham_terrier',
 'n02096051-Airedale',
 'n02108000-EntleBucher',
 'n02091831-Saluki',
 'n02110958-pug',
 'n02095570-Lakeland_terrier',
 'n02096177-cairn',
 'n02086910-papillon',
 'n02091467-Norwegian_elkhound',
 'n02096294-Australian_terrier',
 'n02112137-chow',
 'n02088364-beagle',
 'n02111277-Newfoundland',
 'n02110185-Siberian_husky',
 'n02086646-Blenheim_spaniel',
 'n02091244-Ibizan_hound',
 'n02088466-bloodhound',
 'n02091134-whippet',
 'n02098413-Lhasa',
 'n02085782-Japanese_spaniel',
 'n02094258-Norwich_terrier',
 'n02107312-miniature_pinscher',
 'n02097658-silky_terrier',
 'n02091032-Italian_greyhound',
 'n02093647-Bedlington_terrier',
 'n02096

### Transforms and Load Datasets

In [6]:
transform = transforms.Compose(    
    [transforms.RandomResizedCrop(224),
     transforms.ToTensor(),
     transforms.Normalize((0.438, 0.438, 0.438), (0.237,0.237,0.237))])

batch_size = 4

trainset = torchvision.datasets.ImageFolder(root='./newimages',transform=transform)


### Obtaining all the target classes

In [7]:
target_list = torch.tensor(trainset.targets)

In [8]:
target_list

tensor([  0,   0,   0,  ..., 119, 119, 119])

In [13]:
len(target_list)

15417

In [25]:
target_list[15200:15270]

tensor([117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117,
        117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117,
        117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117,
        117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 118, 118, 118,
        118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118])

In [27]:
class_counts = [i for i in y]
class_weight = 1./torch.tensor(class_counts, dtype=torch.float)

In [34]:
class_counts

[72,
 185,
 252,
 16,
 214,
 188,
 196,
 166,
 164,
 239,
 168,
 195,
 187,
 158,
 132,
 82,
 118,
 14,
 56,
 218,
 182,
 187,
 188,
 196,
 54,
 200,
 232,
 134,
 102,
 140,
 180,
 162,
 172,
 152,
 160,
 185,
 138,
 116,
 197,
 202,
 202,
 197,
 196,
 174,
 178,
 90,
 114,
 100,
 122,
 206,
 183,
 110,
 150,
 186,
 70,
 52,
 42,
 156,
 142,
 68,
 88,
 136,
 98,
 80,
 66,
 40,
 130,
 38,
 128,
 50,
 36,
 34,
 86,
 32,
 30,
 64,
 78,
 84,
 148,
 112,
 76,
 28,
 26,
 62,
 60,
 24,
 184,
 144,
 218,
 48,
 202,
 46,
 108,
 58,
 126,
 106,
 154,
 22,
 170,
 192,
 20,
 209,
 200,
 210,
 195,
 213,
 218,
 219,
 196,
 120,
 74,
 176,
 96,
 44,
 94,
 124,
 92,
 104,
 18,
 146]

### obtaining the class weights for the weighted random sampler

In [9]:
class_count = [i for i in y]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
class_weights

tensor([0.0139, 0.0054, 0.0040, 0.0625, 0.0047, 0.0053, 0.0051, 0.0060, 0.0061,
        0.0042, 0.0060, 0.0051, 0.0053, 0.0063, 0.0076, 0.0122, 0.0085, 0.0714,
        0.0179, 0.0046, 0.0055, 0.0053, 0.0053, 0.0051, 0.0185, 0.0050, 0.0043,
        0.0075, 0.0098, 0.0071, 0.0056, 0.0062, 0.0058, 0.0066, 0.0063, 0.0054,
        0.0072, 0.0086, 0.0051, 0.0050, 0.0050, 0.0051, 0.0051, 0.0057, 0.0056,
        0.0111, 0.0088, 0.0100, 0.0082, 0.0049, 0.0055, 0.0091, 0.0067, 0.0054,
        0.0143, 0.0192, 0.0238, 0.0064, 0.0070, 0.0147, 0.0114, 0.0074, 0.0102,
        0.0125, 0.0152, 0.0250, 0.0077, 0.0263, 0.0078, 0.0200, 0.0278, 0.0294,
        0.0116, 0.0312, 0.0333, 0.0156, 0.0128, 0.0119, 0.0068, 0.0089, 0.0132,
        0.0357, 0.0385, 0.0161, 0.0167, 0.0417, 0.0054, 0.0069, 0.0046, 0.0208,
        0.0050, 0.0217, 0.0093, 0.0172, 0.0079, 0.0094, 0.0065, 0.0455, 0.0059,
        0.0052, 0.0500, 0.0048, 0.0050, 0.0048, 0.0051, 0.0047, 0.0046, 0.0046,
        0.0051, 0.0083, 0.0135, 0.0057, 

In [18]:
class_weights.sum()

tensor(1.4956)

### obtaining weights for each target sample

In [10]:
class_weights_all = class_weights[target_list]
class_weights_all

tensor([0.0139, 0.0139, 0.0139,  ..., 0.0068, 0.0068, 0.0068])

### using the pytorch Weighted random sampler class

In [11]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

### uploading data with pytorch data loader class

In [12]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False, sampler=weighted_sampler)

classes = tuple(sort_key)

### setting default device as GPU

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### downloading pytorch pretrained RESNET

In [14]:
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features


#### Adding a fully connected layer with 120 neurons 
#### Uploading model to GPU
#### Setting optimizer as Adagrad with learning rate 0.01

In [15]:
model_ft.fc = nn.Linear(num_ftrs, 120)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adagrad(model_ft.parameters(), lr=0.01)

### Uploading data to GPU

In [16]:
for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
inputs, labels = data[0].to(device), data[1].to(device)

### Training Model 

In [17]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 5.000
[1,   400] loss: 4.797
[1,   600] loss: 4.803
[1,   800] loss: 4.801
[1,  1000] loss: 4.798
[1,  1200] loss: 4.788
[1,  1400] loss: 4.787
[1,  1600] loss: 4.780
[1,  1800] loss: 4.784
[1,  2000] loss: 4.769
[1,  2200] loss: 4.766
[1,  2400] loss: 4.755
[1,  2600] loss: 4.762
[1,  2800] loss: 4.760
[1,  3000] loss: 4.756
[1,  3200] loss: 4.727
[1,  3400] loss: 4.719
[1,  3600] loss: 4.722
[1,  3800] loss: 4.775
[2,   200] loss: 4.754
[2,   400] loss: 4.756
[2,   600] loss: 4.761
[2,   800] loss: 4.740
[2,  1000] loss: 4.741
[2,  1200] loss: 4.770
[2,  1400] loss: 4.754
[2,  1600] loss: 4.732
[2,  1800] loss: 4.730
[2,  2000] loss: 4.730
[2,  2200] loss: 4.743
[2,  2400] loss: 4.717
[2,  2600] loss: 4.718
[2,  2800] loss: 4.716
[2,  3000] loss: 4.727
[2,  3200] loss: 4.677
[2,  3400] loss: 4.713
[2,  3600] loss: 4.705
[2,  3800] loss: 4.710
Finished Training


### saving model as pkl

In [19]:
from joblib import dump
dump(model_ft, 'trans_learn_Model.pkl')

['trans_learn_Model.pkl']

In [22]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 